# Setup

In [ ]:
import os
OPENAI_API_KEY="your_openai_api_key_here"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# What are Large Language Models (LLMs)?

Large Language Models (LLMs) are artificial intelligence systems trained on vast amounts of text data to understand, generate, and manipulate human language. They use deep learning architectures, particularly transformers, to process and generate text by predicting the most likely next words in a sequence.

**Important Note**: LLMs are trained at a specific point in time using data available up to that moment. As new information, events, and knowledge continue to emerge in the real world, the model's training data becomes increasingly outdated, creating a knowledge gap that grows over time.

## Strengths of LLMs

* 🎯 **Versatility**
* 🧠 **Language Understanding**
* 🚀 **Accessibility**
* 💡 **Creative and Analytical**

## Weaknesses and Limitations

* ❌ **Hallucinations**
* 📅 **Knowledge Cutoff**
* 🎲 **Inconsistency**
* 🔍 **Lack of True Understanding**
* ⚖️ **Bias and Ethics**
* 💰 **Cost and Resources**

## Summary:
* Biggest issues isolation from the real world and memory.


## Best Practices for Using LLMs

1. **Verify Information**: Always fact-check important information
2. **Use Clear Prompts**: Be specific and detailed in your requests
3. **Implement Safeguards**: Add content filtering and validation
4. **Monitor Costs**: Track API usage and implement usage limits
5. **Combine with Other Tools**: Use LLMs as part of larger systems (like we'll see with LangChain agents)

### Most basic interaction with a LLM an isolate call
Using a LLM to suggest an Italian food name

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)
name = llm("I want to open a restaurant for Italian food. Suggest a fancy name for this.")
print(name)

### Using Langchain to create prompt template for differents cuisines

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variable = ["cuisine"],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

prompt_template_name.format(cuisine="Italian")

### What is a Chain?
A chain is a sequence of calls to components like LLMs, prompts, and other chains. It allows you to:

* Link multiple operations together - Instead of making separate, isolated calls
* Pass outputs from one step as inputs to the next - Creating a pipeline of operations
* Build complex workflows - Combine simple operations into sophisticated applications
* Reuse components - Create modular, reusable pieces of functionality

### Using Langchain chain
We are creating a simple chain, ussing the llm we created, the prompt for restaraunt names and a why to identify the output with a key.

In [ ]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")
name_chain.run("American")

### Chain for menus 
Now we are creating another chain for menu suggestions for a restaurant.

In [ ]:
prompt_template_items = PromptTemplate(
    input_variable = ["restaurant_name"],
    template = "Suggest some menu items for {restaurant_name}. Return it as list sepparate by comma."
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")
food_items_chain.run("Mexican")

### Linking chains
Now we want to use both chains to create a menu both using the restaurant name to before.

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
chain.run("Japanesse")

If you notice we only get the menu but not the restaurant name. We want both so we are going to use the output key to return both.

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=["cuisine"],
    output_variables=["restaurant_name", "menu_items"]
)

chain({"cuisine": "Arabic"})

Observation: at this point we should be able to understand the basics of an ai agent interaction that is the linking of multiple operations.

## AI AGENTS
AI Agents were created to solve this isolation problem by giving LLMs the ability to:

* 🔧 Use Tools: Web search (Wikipedia, Google), Calculators for math, Database queries, API calls and File operations. 
* 🧠 Think and Plan: Break down complex problems into steps, Decide which tools to use and when, Chain together multiple actions and Reason about the results
* 🔄 Act and React: Take actions based on information, Get feedback from tools, Adjust their approach based on results and Iterate until they solve the problem

Summay: AI agent give the capabilities to execute tools that can help to fill some of the issues of isolation and to be able to store memory of actions or knowledge.

### Tools
A tool is a function or capability that an agent can use to interact with the external world. Tools are essentially functions with descriptions that:

Perform specific tasks (search Wikipedia, do math, call APIs)
Have clear descriptions so the agent knows when to use them
Return results that the agent can reason about
Can be combined to solve complex problems

In [ ]:
from langchain_core.tools import tool

We are going to use some predifine tools like wikipedia and llm-math, then we are going to create an agent with this tools.

Note: ZERO_SHOT_REACT_DESCRIPTION is a specific agent type in LangChain that defines how the agent thinks and acts.

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # verbose=True
)

agent.run("When was Albert Einstein born? What would be his age right now in 2026?")

We see that the LLM was able to solve the problem?

How?

So, now try to uncomment the `verbose=True` line and run it again.


------------------------

What we see is that the agent would do a react loop where:

* Thought: "I need to find Einstein's birth date and calculate his age"
* Action: Use Wikipedia tool to search "Albert Einstein"
* Observation: "Einstein was born March 14, 1879"
* Thought: "Now I need to calculate 2026 - 1879"
* Action: Use llm-math tool to calculate the difference
* Observation: "The result is 147 years"
* Thought: "I have both pieces of information needed"
* Final Answer: "Albert Einstein was born on March 14, 1879. He would be 147 years old in 2026."

Notes: 
* You probably would notice that the wikipedia information would bring to posible dates, here the LLM creativity would cause something to use one date or the other. 
* Also you probably would notice that the agent do not return the date and the years, only the year. That because the way we wrote the prompt and since the years is the last questions we only see it because of the step by step it does. If we change the prompt for a 1 questions linked with `and` instead of two questions we are going to see differents results.

## Memory

We have been using the same LLM all this time, but does it recall what was the questions and conversations we had?

In [ ]:
last_question = llm("What was my previous question?")
print(last_question)

If we check the chain we can see it have a memory property.

In [ ]:
dir(chain)

In [ ]:
type(chain.memory)

Wich is empty.

But we can set something there. Let's try using a buffer from langchain.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = name_chain.run("French")
print(name)

In [ ]:
name_chain.memory

In [ ]:
print(name_chain.memory.buffer)

This is the way that langchain adds memory to the Chains. 

But how it works? We can follow the next example using a Conversation Chain since I believe most of you have ever use ChatGpt, so it would be easier to understand.

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm)
print(convo.prompt.template)

In [ ]:
convo.run("Who won the last Lacrosse tournament?")

In [ ]:
convo.run("What is 5+5?")

In [ ]:
convo.run("Who was the captain of the winning team?")

In [ ]:
convo.memory.buffer

At the end how the memory works is basically that it adds to the prompt it sents to the LLM the history of the conversation or the additional data stored on the memory to the LLM so it. 

So be carefull with the conversations or context you add since the price can increase depending on the input tokens and the output tokens.